# 📺 Netflix Recommendation System  

This notebook implements a **Netflix Recommendation System** that leverages **Natural Language Processing (NLP)** and **Machine Learning** techniques to suggest movies and TV shows based on user preferences.  

### 🔹 Key Features:  
- Utilizes **NLP techniques** to analyze movie descriptions, genres, and user reviews.  
- Implements **content-based filtering** and **collaborative filtering** to enhance recommendations.  
- Uses **TF-IDF Vectorization**, **Cosine Similarity**, and **Neural Networks** to improve accuracy.  
- Processes **user watch history and preferences** to generate personalized recommendations.  

This system aims to provide an efficient and engaging way for users to discover content tailored to their interests. 🎬✨  


importing  lib 

In [312]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from ydata_profiling import ProfileReport
from sklearn.impute import SimpleImputer

# text preprocessing 
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder


## dorp the model in pickle 
import pickle
import os 

Load the dataset

In [313]:
# Load the dataset (Replace 'songs.csv' with your actual dataset)
nf_movies = pd.read_csv('D:\\NF_Recommendations_ml\\data\\netflix_titles.csv')

In [314]:
nf_movies.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [315]:
# tags:
# type 
# title 
# director 
# cast 
# reslease_year 
# rating 
# description

In [316]:
nf_movies_1 = ["type", "title", "director", "cast", "release_year", "rating", "description"]
nf_movies = nf_movies[nf_movies_1]

In [317]:
nf_movies.head()

,type,title,director,cast,release_year,rating,description
0,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,2020,PG-13,"As her father nears the end of his life, filmm..."
1,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",2021,TV-MA,"After crossing paths at a party, a Cape Town t..."
2,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",2021,TV-MA,To protect his family from a powerful drug lor...
3,TV Show,Jailbirds New Orleans,NaN,NaN,2021,TV-MA,"Feuds, flirtations and toilet talk go down amo..."
4,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",2021,TV-MA,In a city of coaching centers known to train I...


In [318]:
# Create the profile report
profile = ProfileReport(nf_movies, explorative=True)

# Save the report to an HTML file
profile.to_file("output_report.html")

# To display in a Jupyter Notebook
profile.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [319]:
print(nf_movies.shape)

(8807, 7)


In [320]:
nf_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   type          8807 non-null   object
 1   title         8807 non-null   object
 2   director      6173 non-null   object
 3   cast          7982 non-null   object
 4   release_year  8807 non-null   int64 
 5   rating        8803 non-null   object
 6   description   8807 non-null   object
dtypes: int64(1), object(6)
memory usage: 481.8+ KB


In [321]:
print(nf_movies.columns)

Index(['type', 'title', 'director', 'cast', 'release_year', 'rating',
       'description'],
      dtype='object')


In [322]:
null_values = nf_movies.isnull().sum()
print(f"All columns missing Values: {null_values}")
print(f"All columns missing values: {', '.join(null_values.index)}")


All columns missing Values: type               0
title              0
director        2634
cast             825
release_year       0
rating             4
description        0
dtype: int64
All columns missing values: type, title, director, cast, release_year, rating, description


In [323]:
# Calculate missing values percentage
null_percentages = (nf_movies.isnull().sum() / len(nf_movies)) * 100

# Filter only columns with missing values
null_percentages = null_percentages[null_percentages > 0]
print(null_percentages)

director    29.908028
cast         9.367549
rating       0.045418
dtype: float64


In [324]:
from sklearn.impute import SimpleImputer

# Select categorical columns
categorical_cols = ["director", "cast", "rating"]

# Create an imputer with the most frequent value
imputer = SimpleImputer(strategy="most_frequent")

# Apply the imputer to the categorical columns
nf_movies[categorical_cols] = imputer.fit_transform(nf_movies[categorical_cols])

In [325]:
nf_movies.isnull().sum()

type            0
title           0
director        0
cast            0
release_year    0
rating          0
description     0
dtype: int64

In [326]:
# Count total duplicate rows
duplicate_count = nf_movies.duplicated().sum()
print(f"Total duplicate rows: {duplicate_count}")


Total duplicate rows: 0


In [327]:
nf_movies.head()

,type,title,director,cast,release_year,rating,description
0,Movie,Dick Johnson Is Dead,Kirsten Johnson,David Attenborough,2020,PG-13,"As her father nears the end of his life, filmm..."
1,TV Show,Blood & Water,Rajiv Chilaka,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",2021,TV-MA,"After crossing paths at a party, a Cape Town t..."
2,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",2021,TV-MA,To protect his family from a powerful drug lor...
3,TV Show,Jailbirds New Orleans,Rajiv Chilaka,David Attenborough,2021,TV-MA,"Feuds, flirtations and toilet talk go down amo..."
4,TV Show,Kota Factory,Rajiv Chilaka,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",2021,TV-MA,In a city of coaching centers known to train I...


Text preprocessing 

In [328]:
nf_movies['cast']

0                                      David Attenborough
1       Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...
2       Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...
3                                      David Attenborough
4       Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...
                              ...                        
8802    Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...
8803                                   David Attenborough
8804    Jesse Eisenberg, Woody Harrelson, Emma Stone, ...
8805    Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...
8806    Vicky Kaushal, Sarah-Jane Dias, Raaghav Chanan...
Name: cast, Length: 8807, dtype: object

In [329]:
# Extract only the first 3 cast members
nf_movies['cast'] = nf_movies['cast'].apply(lambda x: ', '.join(x.split(', ')[:3]))

In [330]:
# List of specific columns to clean
columns_to_clean = ['cast', 'director']  # Add any columns you want

# Remove all spaces from these columns
nf_movies[columns_to_clean] = nf_movies[columns_to_clean].applymap(lambda x: x.replace(" ", "") if isinstance(x, str) else x)


C:\Users\ASUS\AppData\Local\Temp\ipykernel_10200\2595267707.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  nf_movies[columns_to_clean] = nf_movies[columns_to_clean].applymap(lambda x: x.replace(" ", "") if isinstance(x, str) else x)


In [331]:
nf_movies.head()

,type,title,director,cast,release_year,rating,description
0,Movie,Dick Johnson Is Dead,KirstenJohnson,DavidAttenborough,2020,PG-13,"As her father nears the end of his life, filmm..."
1,TV Show,Blood & Water,RajivChilaka,"AmaQamata,KhosiNgema,GailMabalane",2021,TV-MA,"After crossing paths at a party, a Cape Town t..."
2,TV Show,Ganglands,JulienLeclercq,"SamiBouajila,TracyGotoas,SamuelJouy",2021,TV-MA,To protect his family from a powerful drug lor...
3,TV Show,Jailbirds New Orleans,RajivChilaka,DavidAttenborough,2021,TV-MA,"Feuds, flirtations and toilet talk go down amo..."
4,TV Show,Kota Factory,RajivChilaka,"MayurMore,JitendraKumar,RanjanRaj",2021,TV-MA,In a city of coaching centers known to train I...


In [332]:
nf_movies['description'][123]

'Based on the last book of the epic Ramayana, this series follows the endeavors and adventures of Lord Rama’s twin sons through their childhood.'

In [333]:
nf_movies["description"] = nf_movies["description"].str.lower()


In [334]:
nf_movies.head()

,type,title,director,cast,release_year,rating,description
0,Movie,Dick Johnson Is Dead,KirstenJohnson,DavidAttenborough,2020,PG-13,"as her father nears the end of his life, filmm..."
1,TV Show,Blood & Water,RajivChilaka,"AmaQamata,KhosiNgema,GailMabalane",2021,TV-MA,"after crossing paths at a party, a cape town t..."
2,TV Show,Ganglands,JulienLeclercq,"SamiBouajila,TracyGotoas,SamuelJouy",2021,TV-MA,to protect his family from a powerful drug lor...
3,TV Show,Jailbirds New Orleans,RajivChilaka,DavidAttenborough,2021,TV-MA,"feuds, flirtations and toilet talk go down amo..."
4,TV Show,Kota Factory,RajivChilaka,"MayurMore,JitendraKumar,RanjanRaj",2021,TV-MA,in a city of coaching centers known to train i...


In [335]:
import string
exclude  = string.punctuation
print(exclude)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [336]:
# Remove punctuation from the "description" column
nf_movies['description'] = nf_movies['description'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

In [337]:
nf_movies['description'][100]

'an intergalactic device transforms toy cars into robots the tobots working with friends to solve mysteries they protect the world from evil'

In [338]:
nf_movies.head()

,type,title,director,cast,release_year,rating,description
0,Movie,Dick Johnson Is Dead,KirstenJohnson,DavidAttenborough,2020,PG-13,as her father nears the end of his life filmma...
1,TV Show,Blood & Water,RajivChilaka,"AmaQamata,KhosiNgema,GailMabalane",2021,TV-MA,after crossing paths at a party a cape town te...
2,TV Show,Ganglands,JulienLeclercq,"SamiBouajila,TracyGotoas,SamuelJouy",2021,TV-MA,to protect his family from a powerful drug lor...
3,TV Show,Jailbirds New Orleans,RajivChilaka,DavidAttenborough,2021,TV-MA,feuds flirtations and toilet talk go down amon...
4,TV Show,Kota Factory,RajivChilaka,"MayurMore,JitendraKumar,RanjanRaj",2021,TV-MA,in a city of coaching centers known to train i...


In [339]:


# Download stopwords if not already available
nltk.download('stopwords')

# Get English stopwords list
stop_words = set(stopwords.words('english'))

# Function to remove stopwords
def remove_stopwords(text):
    if isinstance(text, str):  # Ensure it's a string
        return ' '.join([word for word in text.split() if word.lower() not in stop_words])
    return text  # Return as-is if not a string

# Apply function to specific columns
nf_movies['description'] = nf_movies["description"].apply(remove_stopwords)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [340]:
nf_movies.head()

,type,title,director,cast,release_year,rating,description
0,Movie,Dick Johnson Is Dead,KirstenJohnson,DavidAttenborough,2020,PG-13,father nears end life filmmaker kirsten johnso...
1,TV Show,Blood & Water,RajivChilaka,"AmaQamata,KhosiNgema,GailMabalane",2021,TV-MA,crossing paths party cape town teen sets prove...
2,TV Show,Ganglands,JulienLeclercq,"SamiBouajila,TracyGotoas,SamuelJouy",2021,TV-MA,protect family powerful drug lord skilled thie...
3,TV Show,Jailbirds New Orleans,RajivChilaka,DavidAttenborough,2021,TV-MA,feuds flirtations toilet talk go among incarce...
4,TV Show,Kota Factory,RajivChilaka,"MayurMore,JitendraKumar,RanjanRaj",2021,TV-MA,city coaching centers known train india’s fine...


In [341]:
nf_movies['description']

0       father nears end life filmmaker kirsten johnso...
1       crossing paths party cape town teen sets prove...
2       protect family powerful drug lord skilled thie...
3       feuds flirtations toilet talk go among incarce...
4       city coaching centers known train india’s fine...
                              ...                        
8802    political cartoonist crime reporter pair cops ...
8803    living alone spooky town young girl befriends ...
8804    looking survive world taken zombies dorky coll...
8805    dragged civilian life former superhero must tr...
8806    scrappy poor boy worms way tycoons dysfunction...
Name: description, Length: 8807, dtype: object

In [342]:
import nltk
from nltk.tokenize import word_tokenize

# Download required package
nltk.download('punkt')

# Tokenize and clean the descriptions
nf_movies['description'] = nf_movies['description'].astype(str).apply(lambda x: ' '.join(word_tokenize(x)))

# Display the first few rows
print(nf_movies[['description']].head())



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


                                         description
0  father nears end life filmmaker kirsten johnso...
1  crossing paths party cape town teen sets prove...
2  protect family powerful drug lord skilled thie...
3  feuds flirtations toilet talk go among incarce...
4  city coaching centers known train india ’ s fi...


In [343]:
nf_movies['description'][193]

'young private ’ s assignment capture army deserters reveals painful reality endured enlistee compulsory call duty'

In [344]:
# Download required NLTK package
nltk.download('punkt')

# Initialize the Porter Stemmer
stemmer = PorterStemmer()

# Function to apply stemming
def apply_stemming(text):
    tokens = word_tokenize(text)  # Tokenize text
    stemmed_tokens = [stemmer.stem(token) for token in tokens]  # Apply stemming
    return ' '.join(stemmed_tokens)  # Join tokens back into a string

# Apply stemming to the 'description' column
nf_movies['description'] = nf_movies['description'].astype(str).apply(apply_stemming)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [345]:
nf_movies.head()

,type,title,director,cast,release_year,rating,description
0,Movie,Dick Johnson Is Dead,KirstenJohnson,DavidAttenborough,2020,PG-13,father near end life filmmak kirsten johnson s...
1,TV Show,Blood & Water,RajivChilaka,"AmaQamata,KhosiNgema,GailMabalane",2021,TV-MA,cross path parti cape town teen set prove whet...
2,TV Show,Ganglands,JulienLeclercq,"SamiBouajila,TracyGotoas,SamuelJouy",2021,TV-MA,protect famili power drug lord skill thief meh...
3,TV Show,Jailbirds New Orleans,RajivChilaka,DavidAttenborough,2021,TV-MA,feud flirtat toilet talk go among incarcer wom...
4,TV Show,Kota Factory,RajivChilaka,"MayurMore,JitendraKumar,RanjanRaj",2021,TV-MA,citi coach center known train india ’ s finest...


In [346]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
nf_movies['release_year'] = scaler.fit_transform(nf_movies[['release_year']])


In [347]:
nf_movies.head(1)

,type,title,director,cast,release_year,rating,description
0,Movie,Dick Johnson Is Dead,KirstenJohnson,DavidAttenborough,0.989583,PG-13,father near end life filmmak kirsten johnson s...


In [351]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse_output=False)
nf_movies['rating'] = encoder.fit_transform(nf_movies[['rating']])


In [352]:
nf_movies.head()

,type,title,director,cast,release_year,rating,description
0,Movie,Dick Johnson Is Dead,KirstenJohnson,DavidAttenborough,0.989583,0.0,father near end life filmmak kirsten johnson s...
1,TV Show,Blood & Water,RajivChilaka,"AmaQamata,KhosiNgema,GailMabalane",1.000000,0.0,cross path parti cape town teen set prove whet...
2,TV Show,Ganglands,JulienLeclercq,"SamiBouajila,TracyGotoas,SamuelJouy",1.000000,0.0,protect famili power drug lord skill thief meh...
3,TV Show,Jailbirds New Orleans,RajivChilaka,DavidAttenborough,1.000000,0.0,feud flirtat toilet talk go among incarcer wom...
4,TV Show,Kota Factory,RajivChilaka,"MayurMore,JitendraKumar,RanjanRaj",1.000000,0.0,citi coach center known train india ’ s finest...


In [353]:
import pandas as pd

# Convert all relevant columns to strings and merge them into a single 'tags' column
nf_movies['tags'] = (
    nf_movies['description'].astype(str) + " " +
    nf_movies['cast'].astype(str) + " " +
    nf_movies['type'].astype(str) + " " +
    nf_movies['director'].astype(str) + " " +
    nf_movies['rating'].astype(str) + " " +
    nf_movies['release_year'].astype(str)
)

# # Fill any NaN values with empty strings
# nf_movies['tags'] = nf_movies['tags'].fillna("")


In [354]:
nf_movies['tags'][0]

'father near end life filmmak kirsten johnson stage death invent comic way help face inevit DavidAttenborough Movie KirstenJohnson 0.0 0.9895833333333321'

In [355]:
nf_movies.head()

,type,title,director,cast,release_year,rating,description,tags
0,Movie,Dick Johnson Is Dead,KirstenJohnson,DavidAttenborough,0.989583,0.0,father near end life filmmak kirsten johnson s...,father near end life filmmak kirsten johnson s...
1,TV Show,Blood & Water,RajivChilaka,"AmaQamata,KhosiNgema,GailMabalane",1.000000,0.0,cross path parti cape town teen set prove whet...,cross path parti cape town teen set prove whet...
2,TV Show,Ganglands,JulienLeclercq,"SamiBouajila,TracyGotoas,SamuelJouy",1.000000,0.0,protect famili power drug lord skill thief meh...,protect famili power drug lord skill thief meh...
3,TV Show,Jailbirds New Orleans,RajivChilaka,DavidAttenborough,1.000000,0.0,feud flirtat toilet talk go among incarcer wom...,feud flirtat toilet talk go among incarcer wom...
4,TV Show,Kota Factory,RajivChilaka,"MayurMore,JitendraKumar,RanjanRaj",1.000000,0.0,citi coach center known train india ’ s finest...,citi coach center known train india ’ s finest...


In [356]:
# # Drop specified columns
# nf_movies = nf_movies.drop(columns=['description', 'cast', 'type', 'director'], inplace=True)
nf_movies = nf_movies.drop(columns=['type','director','cast','release_year','rating','description'],axis = 1)

In [357]:
nf_movies.head()

,title,tags
0,Dick Johnson Is Dead,father near end life filmmak kirsten johnson s...
1,Blood & Water,cross path parti cape town teen set prove whet...
2,Ganglands,protect famili power drug lord skill thief meh...
3,Jailbirds New Orleans,feud flirtat toilet talk go among incarcer wom...
4,Kota Factory,citi coach center known train india ’ s finest...


In [368]:
nf_movies['tags'][0]

'father near end life filmmak kirsten johnson stage death invent comic way help face inevit DavidAttenborough Movie KirstenJohnson 0.0 0.9895833333333321'

In [358]:
print(nf_movies.columns)

Index(['title', 'tags'], dtype='object')


In [359]:
# Initialize TF-IDF Vectorizer
tfidf = TfidfVectorizer(stop_words='english', max_features=10000, ngram_range=(1, 3), min_df=3, max_df=0.8)


# Apply TF-IDF on the 'tags' column
vector  = tfidf.fit_transform(nf_movies['tags']).toarray()


In [360]:
vector.shape

(8807, 10000)

In [361]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vector)

In [362]:
similarity

array([[1.        , 0.        , 0.        , ..., 0.00319336, 0.02031219,
        0.06431163],
       [0.        , 1.        , 0.007002  , ..., 0.04361809, 0.        ,
        0.        ],
       [0.        , 0.007002  , 1.        , ..., 0.02946748, 0.        ,
        0.01507663],
       ...,
       [0.00319336, 0.04361809, 0.02946748, ..., 1.        , 0.00268816,
        0.0025622 ],
       [0.02031219, 0.        , 0.        , ..., 0.00268816, 1.        ,
        0.00222239],
       [0.06431163, 0.        , 0.01507663, ..., 0.0025622 , 0.00222239,
        1.        ]])

In [363]:
from sklearn.metrics import precision_score, recall_score

# Example: Compare recommended movies with actual watched movies
def precision_recall_at_k(actual, predicted, k=5):
    actual_set = set(actual)
    predicted_set = set(predicted[:k])
    precision = len(actual_set & predicted_set) / k
    recall = len(actual_set & predicted_set) / len(actual_set)
    return precision, recall

# Example usage
actual_movies = ['Inception', 'Interstellar', 'The Matrix']
predicted_movies = ['Inception', 'Avatar', 'The Matrix', 'Titanic', 'Joker']
precision, recall = precision_recall_at_k(actual_movies, predicted_movies, k=5)

print(f'Precision@5: {precision:.2f}')
print(f'Recall@5: {recall:.2f}')


Precision@5: 0.40
Recall@5: 0.67


In [381]:
nf_movies['title'] == "Blood & Water"

0       False
1        True
2       False
3       False
4       False
        ...  
8802    False
8803    False
8804    False
8805    False
8806    False
Name: title, Length: 8807, dtype: bool

In [397]:
# Function to recommend movies
def recommend_movies(title, num_recommendations=5):
    if title not in nf_movies['title'].values:
        return "Movie not found in dataset."
    
    idx = nf_movies[nf_movies['title'] == title].index[0]
   
    similarity_scores = list(enumerate(similarity[idx]))
    # print(f"checking the similarity scores: {similarity_scores}")
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    # print(f"checking the similarity scores: {similarity_scores}")
    similarity_scores = similarity_scores[1:num_recommendations+1]
    # print(similarity_scores)
    
    recommended_movie_indices = [i[0] for i in similarity_scores]
    return nf_movies.iloc[recommended_movie_indices][['title']]
     
# Example usage
print(recommend_movies('Shutter Island'))

                  title
2860               Hugo
6826  Gangs of New York
8272       The Departed
3729           Rakkhosh
6319     Bitter Daisies


In [365]:
%pwd

'd:\\'

In [366]:
os.chdir("../")
%pwd

'd:\\'

In [ ]:
# Create 'model' directory if it doesn't exist
os.makedirs('model', exist_ok=True)

# Save the files inside the 'model' directory
pickle.dump(nf_movies, open('model/movie_data.pkl', 'wb'))
pickle.dump(similarity, open('model/similarity.pkl', 'wb'))

print("Pickle files saved successfully in 'model' directory.")